Start PySpark Session

In [2]:
from pyspark.sql import SparkSession

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034_Preprocessing_1_")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .getOrCreate()
)

22/08/15 12:00:41 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
sdf = spark.read.parquet('../data/raw/tlc_yellow_data/2022-01.parquet')
sdf.show(1, vertical=True, truncate=100)


-RECORD 0------------------------------------
 VendorID              | 1                   
 tpep_pickup_datetime  | 2022-01-01 11:35:40 
 tpep_dropoff_datetime | 2022-01-01 11:53:29 
 passenger_count       | 2.0                 
 trip_distance         | 3.8                 
 RatecodeID            | 1.0                 
 store_and_fwd_flag    | N                   
 PULocationID          | 142                 
 DOLocationID          | 236                 
 payment_type          | 1                   
 fare_amount           | 14.5                
 extra                 | 3.0                 
 mta_tax               | 0.5                 
 tip_amount            | 3.65                
 tolls_amount          | 0.0                 
 improvement_surcharge | 0.3                 
 total_amount          | 21.95               
 congestion_surcharge  | 2.5                 
 airport_fee           | 0.0                 
only showing top 1 row



In [55]:
# Read in NYC Yellow Cab Taxi data from 2020 onwards
sdf_all = spark.read.parquet('../data/raw/tlc_yellow_data/202*')
sdf_all.show(1, vertical=True, truncate=100)

-RECORD 0------------------------------------
 VendorID              | 1                   
 tpep_pickup_datetime  | 2020-01-01 11:28:15 
 tpep_dropoff_datetime | 2020-01-01 11:33:03 
 passenger_count       | 1.0                 
 trip_distance         | 1.2                 
 RatecodeID            | 1.0                 
 store_and_fwd_flag    | N                   
 PULocationID          | 238                 
 DOLocationID          | 239                 
 payment_type          | 1                   
 fare_amount           | 6.0                 
 extra                 | 3.0                 
 mta_tax               | 0.5                 
 tip_amount            | 1.47                
 tolls_amount          | 0.0                 
 improvement_surcharge | 0.3                 
 total_amount          | 11.27               
 congestion_surcharge  | 2.5                 
 airport_fee           | null                
only showing top 1 row



In [56]:
sdf_all.count()
#sdf.count()

68224564

In [57]:
from pyspark.sql import functions as F

In [58]:
# before
sdf_all.printSchema()
# Cast longs to ints
for field in ('PULocationID', 'DOLocationID', 'payment_type', 'RatecodeID', 'VendorID', 'passenger_count'):
    sdf_all = sdf_all.withColumn(
        field,
        F.col(field).cast('INT')
    )

# change airport fee to double
sdf_all = sdf_all.withColumn(
    "airport_fee",
    F.col("airport_fee").cast('INT')
)

sdf_all = sdf_all.withColumn(
    'store_and_fwd_flag',
    (F.col("store_and_fwd_flag") == 'Y').cast('BOOLEAN')
)

# remove airport fee, dunno why it doesnt work
sdf_all = sdf_all.drop('airport_fee')

# after
sdf_all.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: integer (nullable = true)

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestam

In [61]:
sdf_subsample = sdf.sample(fraction=0.1, seed=0)

categorical_features = ["PULocationID", "DOLocationID", "RatecodeID"]
feature_subset = ["tpep_pickup_datetime", "tpep_dropoff_datetime", "passenger_count", "trip_distance", "payment_type", "fare_amount", "extra", "mta_tax", "tip_amount", "tolls_amount", "improvement_surcharge"]

sdf_subsample.describe(feature_subset).show()
#sdf_all.describe().show()


+-------+------------------+-----------------+------------------+------------------+------------------+-------------------+------------------+------------------+---------------------+
|summary|   passenger_count|    trip_distance|      payment_type|       fare_amount|             extra|            mta_tax|        tip_amount|      tolls_amount|improvement_surcharge|
+-------+------------------+-----------------+------------------+------------------+------------------+-------------------+------------------+------------------+---------------------+
|  count|            239066|           246436|            246436|            246436|            246436|             246436|            246436|            246436|               246436|
|   mean|1.3885579714388496|6.114899365352501|1.1934863412813064|12.771523397556553|  1.00607727767047|0.49162865815059487| 2.388714189485343|0.3759088769498818|   0.2968202697671429|
| stddev|0.9798024321445292|638.2008996838017| 0.500443872508818|12.160339892127